# **SPAM CLASSIFICATION USING PYSPARK AND MLLIB**


https://archive.ics.uci.edu/dataset/228/sms+spam+collection - Dataset

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=fbfa2e41d3b9b0a6b20b6e31264bd6fde85620c99d43e7d89ebc634b88a6b194
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("spamclassification").getOrCreate()
spark

In [ ]:
data = spark.read.csv("/content/SMSSpamCollection.csv",inferSchema=True,sep='\t')
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [ ]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



Clean and Prepare the Data:

In [ ]:
from pyspark.sql.functions import length
data = data.withColumn('length',length(data['text']))
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [ ]:
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



Feature Transformations:

In [ ]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [ ]:
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")

In [ ]:
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')

In [ ]:
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')

In [ ]:
idf = IDF(inputCol="c_vec", outputCol="tf_idf")

In [ ]:
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [ ]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [ ]:
from pyspark.ml.linalg import Vector

Model 1 - Naive Bayes

In [ ]:
from pyspark.ml.classification import NaiveBayes
# Use defaults
nb = NaiveBayes()

Pipeline

In [ ]:
from pyspark.ml import Pipeline
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])
cleaner = data_prep_pipe.fit(data)
clean_data = cleaner.transform(data)

Displaying the Output after Pre-processing

In [ ]:
clean_data = clean_data.select(['label','features'])
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,140...|
|  0.0|(13424,[10,53,102...|
|  0.0|(13424,[127,185,4...|
|  1.0|(13424,[1,47,121,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,117...|
|  1.0|(13424,[8,16,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,95,221...|
|  0.0|(13424,[555,1797,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,44...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [ ]:
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [ ]:
spam_predictor_1 = nb.fit(training)

In [ ]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [ ]:
test_results_1 = spam_predictor_1.transform(testing)

In [ ]:
test_results_1.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-794.92333854459...|[1.0,8.5618540511...|       0.0|
|  0.0|(13424,[0,1,2,41,...|[-1063.8127726083...|[1.0,3.7056695773...|       0.0|
|  0.0|(13424,[0,1,5,15,...|[-1001.3649901443...|[1.0,1.1085257959...|       0.0|
|  0.0|(13424,[0,1,5,20,...|[-808.64404748149...|[1.0,9.2022239314...|       0.0|
|  0.0|(13424,[0,1,12,33...|[-448.73357546248...|[1.0,1.5719780526...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-1381.0383114428...|[1.0,2.6382643595...|       0.0|
|  0.0|(13424,[0,1,16,19...|[-806.42074827398...|[1.0,1.3516535644...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-772.07366074171...|[1.0,1.2515505357...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-998.81965190078...|[1.0,1.1104712370...|       0.0|
|  0.0|(13424,[0

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_eval = MulticlassClassificationEvaluator()
acc_1 = acc_eval.evaluate(test_results_1)
print("Accuracy of model at predicting spam was: {}".format(acc_1))

Accuracy of model at predicting spam was: 0.9227910057584431
